In [ ]:
import mysql.connector
from sqlalchemy import create_engine, Column, Integer, String, Float, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base
import pandas as pd
import json
import os

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)


def convert_state_name(input_string):
    # Replace hyphens and special characters with spaces, then capitalize each word
    formatted_string = input_string.replace('-', ' ').replace('&', '&')
    words = formatted_string.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)

# Create a SQLAlchemy engine
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# Path to data
path = "C:\\Users\\HP\\GUVI PROJ\\pulse\\data\\aggregated\\transaction\\country\\india\\state\\"
Agg_state_list = os.listdir(path)

# Prepare the DataFrame
clm = {'State': [], 'Year': [], 'Quater': [], 'Transacion_type': [], 'Transacion_count': [], 'Transacion_amount': []}

for i in Agg_state_list:
    p_i = path + i + "/"
    Agg_yr = os.listdir(p_i)
    for j in Agg_yr:
        p_j = p_i + j + "/"
        Agg_yr_list = os.listdir(p_j)
        for k in Agg_yr_list:
            p_k = p_j + k
            with open(p_k, 'r') as Data:
                D = json.load(Data)
                for z in D['data']['transactionData']:
                    Name = z['name']
                    count = z['paymentInstruments'][0]['count']
                    amount = z['paymentInstruments'][0]['amount']
                    clm['Transacion_type'].append(Name)
                    clm['Transacion_count'].append(count)
                    clm['Transacion_amount'].append(amount)
                    clm['State'].append(i)
                    clm['Year'].append(j)
                    clm['Quater'].append(int(k.strip('.json')))

# Create DataFrame
Agg_Trans = pd.DataFrame(clm)
Agg_Trans['State'] = Agg_Trans['State'].apply(convert_state_name)
# Store DataFrame to MySQL table (ensure the table name is in lowercase)
table_name = 'agg_transaction'
Agg_Trans.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the connection
conn.close()

# Display the table from MySQL
# Reopen the connection using SQLAlchemy for reading the table
conn = engine.connect()

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, con=conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()


In [ ]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# Define the path to the directory containing JSON files
path = "C:\\Users\\HP\\GUVI PROJ\\pulse\\data\\aggregated\\user\\country\\india\\state\\"

# Dictionary to store extracted data
clm_user = {'State': [], 'Year': [], 'Quarter': [], 'Brand_Name': [], 'Num_of_Reg_Users': [], 'Percent_Share': []}

# Function to convert state names for geovisualization
def convert_state_name(input_string):
    # Replace hyphens and special characters with spaces, then capitalize each word
    formatted_string = input_string.replace('-', ' ').replace('&', '&')
    words = formatted_string.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)

# Iterate through each state directory
for state in os.listdir(path):
    state_path = os.path.join(path, state)
    if os.path.isdir(state_path):
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if os.path.isdir(year_path):
                for quarter_file in os.listdir(year_path):
                    quarter_path = os.path.join(year_path, quarter_file)
                    if quarter_file.endswith('.json'):
                        with open(quarter_path, 'r') as file:
                            data = json.load(file)
                            if data['data'] and 'usersByDevice' in data['data'] and data['data']['usersByDevice'] is not None:
                                for device in data['data']['usersByDevice']:
                                    clm_user['State'].append(state)
                                    clm_user['Year'].append(year)
                                    clm_user['Quarter'].append(int(quarter_file.strip('.json')))
                                    clm_user['Brand_Name'].append(device['brand'])
                                    clm_user['Num_of_Reg_Users'].append(device['count'])
                                    clm_user['Percent_Share'].append(device['percentage'])

# Create DataFrame
Agg_User = pd.DataFrame(clm_user)

# Update the 'State' column to be suitable for geovisualization
Agg_User['State'] = Agg_User['State'].apply(convert_state_name)

# Store the updated DataFrame into the MySQL table
table_name = 'agg_user'
Agg_User.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the initial MySQL connection
conn.close()

# Reopen the connection using SQLAlchemy for reading the table
conn = engine.connect()

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, con=conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()


In [ ]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# Define the path to the directory containing JSON files
path = "C:\\Users\\HP\\GUVI PROJ\\pulse\\data\\map\\transaction\\hover\\country\\india\\state\\"

# Dictionary to store extracted data
clm_map_ag = {'State': [], 'Year': [], 'Quarter': [], 'District': [], 'Num_of_Transactions_in_that_Yr': [], 'Total_Trans_iin_that_year': []}

# Function to convert state names for geovisualization
def convert_state_name(input_string):
    # Replace hyphens and special characters with spaces, then capitalize each word
    formatted_string = input_string.replace('-', ' ').replace('&', '&')
    words = formatted_string.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)

# Iterate through each state directory
for state in os.listdir(path):
    state_path = os.path.join(path, state)
    if os.path.isdir(state_path):
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if os.path.isdir(year_path):
                for quarter_file in os.listdir(year_path):
                    quarter_path = os.path.join(year_path, quarter_file)
                    if quarter_file.endswith('.json'):
                        with open(quarter_path, 'r') as file:
                            data = json.load(file)
                            for item in data['data']['hoverDataList']:
                                name = item['name']
                                metric = item['metric'][0]
                                count = metric['count']
                                amount = metric['amount']
                                clm_map_ag['State'].append(state)
                                clm_map_ag['Year'].append(year)
                                clm_map_ag['Quarter'].append(int(quarter_file.strip('.json')))
                                clm_map_ag['District'].append(name)
                                clm_map_ag['Num_of_Transactions_in_that_Yr'].append(count)
                                clm_map_ag['Total_Trans_iin_that_year'].append(amount)
                            

# Create DataFrame
Map_Trans = pd.DataFrame(clm_map_ag)

# Update the 'State' column to be suitable for geovisualization
Map_Trans['State'] = Map_Trans['State'].apply(convert_state_name)
Map_Trans['District'] = Map_Trans['District'].apply(convert_state_name)
# Store the updated DataFrame into the MySQL table
table_name = 'map_trans'
Map_Trans.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the initial MySQL connection
conn.close()

# Reopen the connection using SQLAlchemy for reading the table
conn = engine.connect()

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, con=conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()


In [ ]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")


def convert_state_name(input_string):
    # Replace hyphens and special characters with spaces, then capitalize each word
    formatted_string = input_string.replace('-', ' ').replace('&', '&')
    words = formatted_string.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)



path="C:\\Users\\HP\\GUVI PROJ\\pulse\\data\\map\\user\\hover\\country\\india\\state\\"
clm_map_user = {'State': [], 'Dist_Name':[], 'Year': [], 'Quarter': [], 'Num_of_reg_users': [], 'Num_of_app_opens': []}
for state in os.listdir(path):
    state_path = os.path.join(path, state)
    if os.path.isdir(state_path):
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if os.path.isdir(year_path):
                for quarter_file in os.listdir(year_path):
                    quarter_path = os.path.join(year_path, quarter_file)
                    if quarter_file.endswith('.json'):
                        with open(quarter_path, 'r') as file:
                            data = json.load(file)
                            
                            hover_data = data['data']['hoverData']
                            for district, metrics in hover_data.items():
                                reg_users = metrics['registeredUsers']
                                app_opens = metrics['appOpens']
                                dt_name=district
                                clm_map_user['State'].append(state)
                                clm_map_user['Year'].append(year)
                                clm_map_user['Dist_Name'].append(dt_name)
                                clm_map_user['Quarter'].append(int(quarter_file.strip('.json')))
                                clm_map_user['Num_of_reg_users'].append(reg_users)
                                clm_map_user['Num_of_app_opens'].append(app_opens)
                            
      

                               
Map_User = pd.DataFrame(clm_map_user)

# Update the 'State' column to be suitable for geovisualization
Map_User['State'] = Map_User['State'].apply(convert_state_name)
Map_User['Dist_Name'] = Map_User['Dist_Name'].apply(convert_state_name)
#Map_User['Num_of_reg_users'] = Map_Trans['Num_of_reg_users'].apply(convert_state_name)                               


table_name = 'map_user'
Map_User.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the initial MySQL connection
conn.close()

# Reopen the connection using SQLAlchemy for reading the table
conn = engine.connect()

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, con=conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()



In [ ]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

def convert_state_name(input_string):
    # Replace hyphens and special characters with spaces, then capitalize each word
    formatted_string = input_string.replace('-', ' ').replace('&', '&')
    words = formatted_string.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)

path = "C:\\Users\\HP\\GUVI PROJ\\pulse\\data\\top\\transaction\\country\\india\\state\\"
clm_top_trans_state = {'State': [], 'Year': [], 'Quarter': [], 'District': [], 'Tot_No_of_Trans': [], 'Tot_Val_of_Trans': []}

for state in os.listdir(path):
    state_path = os.path.join(path, state)
    if os.path.isdir(state_path):
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if os.path.isdir(year_path):
                for quarter_file in os.listdir(year_path):
                    quarter_path = os.path.join(year_path, quarter_file)
                    if quarter_file.endswith('.json'):
                        with open(quarter_path, 'r') as file:
                            data = json.load(file)
                            
                            # Check if 'data' key exists
                            if 'data' in data:
                                districts_data = data['data'].get('districts', [])
                                
                                for district_data in districts_data:
                                    entity_name = state
                                    district_name = district_data['entityName']
                                    metric = district_data['metric']
                                    count = metric['count']
                                    amount = metric['amount']
                                    clm_top_trans_state['State'].append(entity_name)
                                    clm_top_trans_state['Year'].append(year)
                                    clm_top_trans_state['Quarter'].append(int(quarter_file.strip('.json')))
                                    clm_top_trans_state['District'].append(district_name)
                                    clm_top_trans_state['Tot_No_of_Trans'].append(count)
                                    clm_top_trans_state['Tot_Val_of_Trans'].append(amount)

# Create DataFrame
Top_Trans_State = pd.DataFrame(clm_top_trans_state)

# Update the 'State' column to be suitable for geovisualization
Top_Trans_State['State'] = Top_Trans_State['State'].apply(convert_state_name)
Top_Trans_State['District'] = Top_Trans_State['District'].apply(convert_state_name)

table_name = 'top_trans_state'
Top_Trans_State.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the initial MySQL connection
conn.close()

# Reopen the connection using SQLAlchemy for reading the table
conn = engine.connect()

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, con=conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()


In [ ]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

def convert_state_name(input_string):
    # Replace hyphens and special characters with spaces, then capitalize each word
    formatted_string = input_string.replace('-', ' ').replace('&', '&')
    words = formatted_string.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)

path = "C:\\Users\\HP\\GUVI PROJ\\pulse\\data\\top\\transaction\\country\\india\\state\\"
clm_top_trans_dist = {'District_Name': [],'Year': [], 'Quarter': [],  'Tot_No_of_DTrans': [], 'Tot_Val_of_DTrans': []}

for state in os.listdir(path):
    state_path = os.path.join(path, state)
    if os.path.isdir(state_path):
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if os.path.isdir(year_path):
                for quarter_file in os.listdir(year_path):
                    quarter_path = os.path.join(year_path, quarter_file)
                    if quarter_file.endswith('.json'):
                        with open(quarter_path, 'r') as file:
                            data = json.load(file)
                            
                            # Check if 'data' key exists
                            if 'data' in data:
                                districts = data['data']['districts']
                                
                                for district in districts:
                                    entity_name = district['entityName']                                 
                                    metric = district['metric']
                                    count = metric['count']
                                    amount = metric['amount']                               
                                    clm_top_trans_dist['Year'].append(year)
                                    clm_top_trans_dist['Quarter'].append(int(quarter_file.strip('.json')))
                                    clm_top_trans_dist['District_Name'].append(district_name)
                                    clm_top_trans_dist['Tot_No_of_DTrans'].append(count)
                                    clm_top_trans_dist['Tot_Val_of_DTrans'].append(amount)

# Create DataFrame
Top_Trans_Dist = pd.DataFrame(clm_top_trans_dist)

# Update the 'State' column to be suitable for geovisualization
Top_Trans_Dist['District_Name'] = Top_Trans_Dist['District_Name'].apply(convert_state_name)

table_name = 'top_trans_dist'
Top_Trans_Dist.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the initial MySQL connection
conn.close()

# Reopen the connection using SQLAlchemy for reading the table
conn = engine.connect()

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, con=conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()


In [ ]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

def convert_state_name(input_string):
    # Replace hyphens and special characters with spaces, then capitalize each word
    formatted_string = input_string.replace('-', ' ').replace('&', '&')
    words = formatted_string.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)

path = "C:\\Users\\HP\\GUVI PROJ\\pulse\\data\\top\\transaction\\country\\india\\state\\"
clm_top_trans_pin = {'Pincode': [], 'State':[],'Year': [], 'Quarter': [],  'Tot_No_of_PTrans': [], 'Tot_Val_of_PTrans': []}

for state in os.listdir(path):
    state_path = os.path.join(path, state)
    if os.path.isdir(state_path):
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if os.path.isdir(year_path):
                for quarter_file in os.listdir(year_path):
                    quarter_path = os.path.join(year_path, quarter_file)
                    if quarter_file.endswith('.json'):
                        with open(quarter_path, 'r') as file:
                            data = json.load(file)
                            
                            # Check if 'data' key exists
                            if 'data' in data and 'pincodes' in data['data']:
                                pincodes = data['data']['pincodes']

                            for pincode in pincodes:
                                entity_name = pincode['entityName']
                                metric = pincode['metric']
                                count = metric['count']
                                amount = metric['amount']
                                clm_top_trans_pin['Year'].append(year)
                                clm_top_trans_pin['Quarter'].append(int(quarter_file.strip('.json')))
                                clm_top_trans_pin['Pincode'].append(entity_name)
                                clm_top_trans_pin['Tot_No_of_PTrans'].append(count)
                                clm_top_trans_pin['Tot_Val_of_PTrans'].append(amount)
                                clm_top_trans_pin['State'].append(state)

# Create DataFrame
Top_Trans_Pin = pd.DataFrame(clm_top_trans_pin)

# Update the 'State' column to be suitable for geovisualization


table_name = 'top_trans_pin'
Top_Trans_Pin['State']=Top_Trans_Pin['State'].apply(convert_state_name)
Top_Trans_Pin.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the initial MySQL connection
conn.close()

# Reopen the connection using SQLAlchemy for reading the table
conn = engine.connect()

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, con=conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()


In [ ]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

def convert_state_name(input_string):
    # Replace hyphens and special characters with spaces, then capitalize each word
    formatted_string = input_string.replace('-', ' ').replace('&', '&')
    words = formatted_string.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)





path="C:\\Users\HP\\GUVI PROJ\\pulse\\data\\top\\user\\country\\india\\state"
clm_top_user_dis = {'District_Name': [],'State':[],'Year': [], 'Quarter': [],  'Reg_Users': []}

for state in os.listdir(path):
    state_path = os.path.join(path, state)
    if os.path.isdir(state_path):
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if os.path.isdir(year_path):
                for quarter_file in os.listdir(year_path):
                    quarter_path = os.path.join(year_path, quarter_file)
                    if quarter_file.endswith('.json'):
                        with open(quarter_path, 'r') as file:
                            data = json.load(file)

                            if 'data' in data and 'districts' in data['data']:
                                districts = data['data']['districts']

                            for dis in districts:
                                name = dis['name']
                                reg_users = dis['registeredUsers']                               
                                clm_top_user_dis['Year'].append(year)
                                clm_top_user_dis['Quarter'].append(int(quarter_file.strip('.json')))
                                clm_top_user_dis['District_Name'].append(name)
                                clm_top_user_dis['State'].append(state)
                                clm_top_user_dis['Reg_Users'].append(reg_users)
                                
# Create DataFrame
Top_User_dis = pd.DataFrame(clm_top_user_dis)

# Update the 'State' column to be suitable for geovisualization


table_name = 'top_user_dis'
Top_User_dis['State'] = Top_User_dis['State'].apply(convert_state_name)
Top_User_dis['District_Name'] = Top_User_dis['District_Name'].apply(convert_state_name)
Top_User_dis.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the initial MySQL connection
conn.close()

# Reopen the connection using SQLAlchemy for reading the table
conn = engine.connect()

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, con=conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()
                            

In [ ]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

def convert_state_name(input_string):
    # Replace hyphens and special characters with spaces, then capitalize each word
    formatted_string = input_string.replace('-', ' ').replace('&', '&')
    words = formatted_string.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)





path="C:\\Users\HP\\GUVI PROJ\\pulse\\data\\top\\user\\country\\india\\state"
clm_top_user_pin = {'Pin_Code': [],'State':[],'Year': [], 'Quarter': [],  'Reg_Users': []}

for state in os.listdir(path):
    state_path = os.path.join(path, state)
    if os.path.isdir(state_path):
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if os.path.isdir(year_path):
                for quarter_file in os.listdir(year_path):
                    quarter_path = os.path.join(year_path, quarter_file)
                    if quarter_file.endswith('.json'):
                        with open(quarter_path, 'r') as file:
                            data = json.load(file)

                            if 'data' in data and 'pincodes' in data['data']:
                                pincodes = data['data']['pincodes']

                            for pin in pincodes:
                                code= pin['name']
                                reg_users = pin['registeredUsers']                               
                                clm_top_user_pin['Year'].append(year)
                                clm_top_user_pin['Quarter'].append(int(quarter_file.strip('.json')))
                                clm_top_user_pin['Pin_Code'].append(code)
                                clm_top_user_pin['Reg_Users'].append(reg_users)
                                clm_top_user_pin['State'].append(state)
                                
# Create DataFrame
Top_User_pin = pd.DataFrame(clm_top_user_pin)

# Update the 'State' column to be suitable for geovisualization


table_name = 'top_user_pin'
Top_User_pin['State']=Top_User_pin['State'].apply(convert_state_name)
Top_User_pin.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the initial MySQL connection
conn.close()

# Reopen the connection using SQLAlchemy for reading the table
conn = engine.connect()

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, con=conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()
                            

In [1]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")
conn = engine.connect()
path_1= "C:\\Users\\HP\\GUVI PROJ\\pulse\\data\\aggregated\\transaction\\country\\india\\state\\"
table_name = 'agg_transaction'

df = pd.read_sql_table(table_name, con=conn)
print(df)

conn.close()

                          State  Year  Quater           Transacion_type  \
0     Andaman & Nicobar Islands  2018       1  Recharge & bill payments   
1     Andaman & Nicobar Islands  2018       1     Peer-to-peer payments   
2     Andaman & Nicobar Islands  2018       1         Merchant payments   
3     Andaman & Nicobar Islands  2018       1        Financial Services   
4     Andaman & Nicobar Islands  2018       1                    Others   
...                         ...   ...     ...                       ...   
4489                West Bengal  2024       1         Merchant payments   
4490                West Bengal  2024       1     Peer-to-peer payments   
4491                West Bengal  2024       1  Recharge & bill payments   
4492                West Bengal  2024       1        Financial Services   
4493                West Bengal  2024       1                    Others   

      Transacion_count  Transacion_amount  
0                 4200       1.845307e+06  
1          

In [2]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")
conn = engine.connect()

table_name = 'agg_user'



df = pd.read_sql_table(table_name, con=conn)
print(df)

conn.close()

                          State  Year  Quarter Brand_Name  Num_of_Reg_Users  \
0     Andaman & Nicobar Islands  2018        1     Xiaomi              1665   
1     Andaman & Nicobar Islands  2018        1    Samsung              1445   
2     Andaman & Nicobar Islands  2018        1       Vivo               982   
3     Andaman & Nicobar Islands  2018        1       Oppo               501   
4     Andaman & Nicobar Islands  2018        1    OnePlus               332   
...                         ...   ...      ...        ...               ...   
6727                West Bengal  2022        1     Lenovo            330017   
6728                West Bengal  2022        1    Infinix            284678   
6729                West Bengal  2022        1       Asus            280347   
6730                West Bengal  2022        1      Apple            277752   
6731                West Bengal  2022        1     Others           2196334   

      Percent_Share  
0          0.247033  
1      

In [1]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")
conn = engine.connect()

table_name = 'map_trans'



df = pd.read_sql_table(table_name, con=conn)
print(df)

conn.close()

                           State  Year  Quarter  \
0      Andaman & Nicobar Islands  2018        1   
1      Andaman & Nicobar Islands  2018        1   
2      Andaman & Nicobar Islands  2018        1   
3      Andaman & Nicobar Islands  2018        2   
4      Andaman & Nicobar Islands  2018        2   
...                          ...   ...      ...   
18291                West Bengal  2024        1   
18292                West Bengal  2024        1   
18293                West Bengal  2024        1   
18294                West Bengal  2024        1   
18295                West Bengal  2024        1   

                                District  Num_of_Transactions_in_that_Yr  \
0      North And Middle Andaman District                             442   
1                 South Andaman District                            5688   
2                      Nicobars District                             528   
3      North And Middle Andaman District                             825   
4      

In [2]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")
conn = engine.connect()

table_name = 'map_user'



df = pd.read_sql_table(table_name, con=conn)
print(df)

conn.close()

                           State                          Dist_Name  Year  \
0      Andaman & Nicobar Islands  North And Middle Andaman District  2018   
1      Andaman & Nicobar Islands             South Andaman District  2018   
2      Andaman & Nicobar Islands                  Nicobars District  2018   
3      Andaman & Nicobar Islands  North And Middle Andaman District  2018   
4      Andaman & Nicobar Islands             South Andaman District  2018   
...                          ...                                ...   ...   
18295                West Bengal                     Nadia District  2024   
18296                West Bengal                   Birbhum District  2024   
18297                West Bengal           Purba Medinipur District  2024   
18298                West Bengal                    Maldah District  2024   
18299                West Bengal                 Darjiling District  2024   

       Quarter  Num_of_reg_users  Num_of_app_opens  
0            1        

In [5]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")
conn = engine.connect()

table_name = 'top_trans_state'



df = pd.read_sql_table(table_name, con=conn)
print(df)

conn.close()

                          State  Year  Quarter                  District  \
0     Andaman & Nicobar Islands  2018        1             South Andaman   
1     Andaman & Nicobar Islands  2018        1                  Nicobars   
2     Andaman & Nicobar Islands  2018        1  North And Middle Andaman   
3     Andaman & Nicobar Islands  2018        2             South Andaman   
4     Andaman & Nicobar Islands  2018        2                  Nicobars   
...                         ...   ...      ...                       ...   
7395                West Bengal  2024        1                    Maldah   
7396                West Bengal  2024        1                     Nadia   
7397                West Bengal  2024        1                    Howrah   
7398                West Bengal  2024        1                   Hooghly   
7399                West Bengal  2024        1         Paschim Medinipur   

      Tot_No_of_Trans  Tot_Val_of_Trans  
0                5688      1.256025e+07  
1  

In [6]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")
conn = engine.connect()

table_name = 'top_trans_dist'



df = pd.read_sql_table(table_name, con=conn)
print(df)

conn.close()

                 District_Name  Year  Quarter  Tot_No_of_DTrans  \
0                South Andaman  2018        1              5688   
1                     Nicobars  2018        1               528   
2     North And Middle Andaman  2018        1               442   
3                South Andaman  2018        2              9395   
4                     Nicobars  2018        2              1120   
...                        ...   ...      ...               ...   
7395                    Maldah  2024        1          42299563   
7396                     Nadia  2024        1          42154481   
7397                    Howrah  2024        1          41536962   
7398                   Hooghly  2024        1          40874089   
7399         Paschim Medinipur  2024        1          40845488   

      Tot_Val_of_DTrans  
0          1.256025e+07  
1          1.139849e+06  
2          9.316631e+05  
3          2.394824e+07  
4          3.072437e+06  
...                 ...  
7395       7.

In [10]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")
conn = engine.connect()

table_name = 'top_trans_pin'



df = pd.read_sql_table(table_name, con=conn)
print(df)

conn.close()

     Pincode                      State  Year  Quarter  Tot_No_of_PTrans  \
0     744101  Andaman & Nicobar Islands  2018        1              1622   
1     744103  Andaman & Nicobar Islands  2018        1              1223   
2     744102  Andaman & Nicobar Islands  2018        1               969   
3     744105  Andaman & Nicobar Islands  2018        1               685   
4     744104  Andaman & Nicobar Islands  2018        1               340   
...      ...                        ...   ...      ...               ...   
8919  732125                West Bengal  2024        1           4490229   
8920  700006                West Bengal  2024        1           4358390   
8921  700001                West Bengal  2024        1           4321119   
8922  732201                West Bengal  2024        1           4265219   
8923  742202                West Bengal  2024        1           3985791   

      Tot_Val_of_PTrans  
0          2.769298e+06  
1          2.238042e+06  
2        

In [8]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")
conn = engine.connect()

table_name = 'top_user_dis'



df = pd.read_sql_table(table_name, con=conn)
print(df)

conn.close()

                 District_Name                      State  Year  Quarter  \
0                South Andaman  Andaman & Nicobar Islands  2018        1   
1     North And Middle Andaman  Andaman & Nicobar Islands  2018        1   
2                     Nicobars  Andaman & Nicobar Islands  2018        1   
3                South Andaman  Andaman & Nicobar Islands  2018        2   
4     North And Middle Andaman  Andaman & Nicobar Islands  2018        2   
...                        ...                        ...   ...      ...   
7395                    Howrah                West Bengal  2024        1   
7396                     Nadia                West Bengal  2024        1   
7397           Purba Medinipur                West Bengal  2024        1   
7398         Paschim Medinipur                West Bengal  2024        1   
7399           Purba Bardhaman                West Bengal  2024        1   

      Reg_Users  
0          5846  
1           632  
2           262  
3          8143

In [9]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")
conn = engine.connect()

table_name = 'top_user_pin'



df = pd.read_sql_table(table_name, con=conn)
print(df)

conn.close()

     Pin_Code                      State  Year  Quarter  Reg_Users
0      744103  Andaman & Nicobar Islands  2018        1       1608
1      744101  Andaman & Nicobar Islands  2018        1       1108
2      744105  Andaman & Nicobar Islands  2018        1       1075
3      744102  Andaman & Nicobar Islands  2018        1       1006
4      744104  Andaman & Nicobar Islands  2018        1        272
...       ...                        ...   ...      ...        ...
8920   700015                West Bengal  2024        1     131676
8921   742304                West Bengal  2024        1     127917
8922   721101                West Bengal  2024        1     127891
8923   700150                West Bengal  2024        1     123919
8924   700091                West Bengal  2024        1     122985

[8925 rows x 5 columns]


In [ ]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")
conn = engine.connect()

table_name_1 = 'map_trans'
table_name_2='map_user'
table_name_3='agg_transaction'
table_name_4='agg_user'
table_name_5='top_trans_state'
table_name_6='top_trans_dis'
table_name_7='top_trans_pin'
table_name_8='top_user_dis'
table_name_9='top_user_pin'



df_1 = pd.read_sql_table(table_name_1, con=conn)
df_2 = pd.read_sql_table(table_name_2, con=conn)
df_3 = pd.read_sql_table(table_name_3, con=conn)
df_4 = pd.read_sql_table(table_name_4, con=conn)
df_5 = pd.read_sql_table(table_name_5, con=conn)
df_6 = pd.read_sql_table(table_name_6, con=conn)
df_7 = pd.read_sql_table(table_name_7, con=conn)
df_8 = pd.read_sql_table(table_name_8, con=conn)
df_9 = pd.read_sql_table(table_name_9, con=conn)



conn.close()

In [3]:
import mysql.connector


import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details

# Establish MySQL database connection
client = mysql.connector.connect(
host = 'localhost',
user = 'root',
password = '1611',
database = 'PhonePePulse_DVandExp'
)

# Create a cursor object to execute SQL queries
cursor = client.cursor()

cursor.execute("""
SELECT 
    t1.Transacion_type,
    t1.State,
    t1.Transacion_count
FROM 
    table1 agg_transaction
JOIN (
    SELECT 
        Transacion_type,
        MAX(Transacion_count) AS Max_Transacion_count
    FROM 
        agg_transaction
    GROUP BY 
        Transacion_type
) t2
ON 
    agg_tra.Transacion_type = t2.Transacion_type AND
    t1.Transacion_count = t2.Max_Transacion_count
ORDER BY 
    t1.Transacion_type;

)
""")
result = cursor.fetchall()
print(result)

ProgrammingError: 1248 (42000): Every derived table must have its own alias